# Simple AI Agent for PTO Hours

This notebook demonstrates a simple AI agent that uses if-else statements to handle PTO (Paid Time Off) queries using OpenAI.


In [ ]:
# Import required libraries
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("openai_api_key"))

# PTO data
PTO_HOURS_AVAILABLE = 13


In [ ]:
# Simple tool function using if-else logic
def get_pto_info(query_type):
    """
    Simple function to get PTO information based on query type
    Uses if-else statements to handle different queries
    """
    if query_type == "available_hours":
        return f"You have {PTO_HOURS_AVAILABLE} PTO hours available."
    elif query_type == "available_days":
        days = PTO_HOURS_AVAILABLE / 8
        return f"You have {days} PTO days available ({PTO_HOURS_AVAILABLE} hours)."
    elif query_type == "can_take_day":
        if PTO_HOURS_AVAILABLE >= 8:
            return f"Yes, you can take a full day off. You have {PTO_HOURS_AVAILABLE} hours available."
        else:
            return f"No, you don't have enough PTO for a full day. You only have {PTO_HOURS_AVAILABLE} hours available."
    elif query_type == "can_take_half_day":
        if PTO_HOURS_AVAILABLE >= 4:
            return f"Yes, you can take a half day off. You have {PTO_HOURS_AVAILABLE} hours available."
        else:
            return f"No, you don't have enough PTO for a half day. You only have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"You have {PTO_HOURS_AVAILABLE} PTO hours available. You can ask about available hours, days, or if you can take time off."


In [ ]:
# Define the tool for OpenAI function calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_pto_info",
            "description": "Get information about PTO (Paid Time Off) hours",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_type": {
                        "type": "string",
                        "enum": ["available_hours", "available_days", "can_take_day", "can_take_half_day", "general"],
                        "description": "The type of PTO information requested"
                    }
                },
                "required": ["query_type"]
            }
        }
    }
]


In [ ]:
# Simple AI Agent with if-else logic
def pto_agent(user_query):
    """
    Simple AI agent that handles PTO queries using if-else statements
    """
    print(f"User: {user_query}\n")
    
    # Create messages for the conversation
    messages = [
        {
            "role": "system",
            "content": "You are a helpful HR assistant that helps employees check their PTO (Paid Time Off) hours. Be friendly and concise."
        },
        {
            "role": "user",
            "content": user_query
        }
    ]
    
    # Get response from OpenAI
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    # Check if the model wants to call a function using if-else
    if response_message.tool_calls:
        # Handle tool calls
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        
        if function_name == "get_pto_info":
            # Parse arguments
            import json
            function_args = json.loads(tool_call.function.arguments)
            query_type = function_args.get("query_type", "general")
            
            # Call our function with if-else logic
            function_response = get_pto_info(query_type)
            
            # Add the function response to messages
            messages.append(response_message)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": function_response
            })
            
            # Get final response from the model
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            
            agent_response = final_response.choices[0].message.content
        else:
            agent_response = "I'm sorry, I can only help with PTO-related queries."
    else:
        # If no tool call, use the direct response
        agent_response = response_message.content
    
    print(f"Agent: {agent_response}\n")
    return agent_response


## Example Usage

Let's test the agent with different types of queries:


In [ ]:
# Example 1: Check available PTO hours
pto_agent("How many PTO hours do I have available?")


In [ ]:
# Example 2: Check available PTO days
pto_agent("How many days of PTO can I take?")


In [ ]:
# Example 3: Check if can take a full day off
pto_agent("Can I take a full day off next week?")


In [ ]:
# Example 4: Check if can take a half day
pto_agent("Do I have enough PTO for a half day?")


## How the If-Else Logic Works

This simple AI agent uses if-else statements in two key places:

1. **In the `get_pto_info()` function**: Different if-else branches handle different query types:
   - `available_hours`: Returns total PTO hours
   - `available_days`: Converts hours to days
   - `can_take_day`: Checks if >= 8 hours available
   - `can_take_half_day`: Checks if >= 4 hours available

2. **In the `pto_agent()` function**: Uses if-else to determine:
   - If OpenAI wants to call a tool (function)
   - Which function to call
   - How to format the final response

The agent leverages OpenAI's intelligence to understand user intent and map it to the appropriate query type, while the actual logic is handled by simple if-else statements.
